In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


## Introduction

Gurgaon, officially named Gurugram, is a city located in the northern Indian state of Haryana. It is situated near the Delhi-Haryana border, about 30 kilometres southwest of the national capital New Delhi It is one of the major satellite cities of Delhi and is part of the National Capital Region of India. Gurgaon has a population of about 10 lacs.
Gurgaon has become a leading financial and industrial hub with the third-highest per capita income in India.Gurgaon has local offices of so many Fortune 500 companies. 

For many shoppers, visiting shopping malls is a great way to relax and enjoy themselves during 
weekends and holidays. Shopping malls are like a one-stop place for all. Real estate developers are also taking advantage of this trend to build more shopping malls to cater to the demand. As a result, there are many shopping malls already in the city. Opening shopping malls allows property developers to earn consistent rental income. The location of the shopping mall is one of the most key question in-front of real estate developers, which will determine if the mall will be a success or a failure.  

So in this project we will try to build a solution which will help answering this question.


#### About Information/Data required for this project

To explore we will need the following data which is publicly available Data: 

1. Gurgaon city Neighbourhoods: https://nominatim.openstreetmap.org/details.php?place_id=263703796 
2. Foursquare Developers Access to venue data: https://foursquare.com/ 

Using this data, we will do exploration first and try to find answer our question. The neighbourhood data will enable us to properly group shopping malls by neighbourhood. 



#### setup/import all required libraries 

In [3]:
#conda search beautifulsoup4 --channel conda-forge
!conda install -c conda-forge beautifulsoup4=4.8.1 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4=4.8.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         207 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.1-py36_0 conda-forge


soupsieve-1.9.

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [6]:
CLIENT_ID = <<  Foursquare ID >>
CLIENT_SECRET = << Foursquare Secret >>
VERSION = '20190905'
LIMIT= '100'

In [7]:
#store retrieval info in a dataframe
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
#sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    #row_categories
    return row_categories_sorted.index.values[0:num_top_venues]

#### Methodology 

At the start, we need to get the list of neighbourhoods of Gurgaon city, though there are many public sites which has these data , we used the list from available in the Nominatim.org site (https://nominatim.openstreetmap.org/details.php?place_id=263703796 ). 
We will do web scraping using Python requests and beautifulsoup packages to extract the list of neighbourhoods data. However, there are many more columns apart from list of names on this page, as well so many different info(in rows) which are not relevant, hence we have to performed data cleaning activities .
Now after getting the list of neighbourhoods, we need to get the geographical coordinates (latitude and longitude) for each neighbourhood in order to be able to use Foursquare API to gather venues data. To do so, we will use the Geocoder package that will allow us to convert address into geographical coordinates in the form of latitude and longitude. After gathering the data, we will populate the data into a pandas DataFrame and then visualize the neighbourhoods in a map using Folium package. 
This allows us to perform a sanity check to make sure that the geographical coordinates data returned by Geocoder are correctly plotted for Gurgaon city. 

In [9]:
address = 'Gurgaon, 122001' # Find Gurgaon Lat/Long to plot the same on map.
geolocator = Nominatim()
location = geolocator.geocode(address, timeout=5)
latitude = location.latitude
longitude = location.longitude
print('Lat and Long of Gurgaon are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


Lat and Long of Gurgaon are 28.4646148, 77.0299194.


In [10]:
page1 = requests.get("https://nominatim.openstreetmap.org/details.php?place_id=263703796")
soup1 = BeautifulSoup(page1.content, 'html.parser')
table1=soup1.find("table",class_="table table-striped table-responsive")#.find_previous("table")
headings1 = [th.get_text() for th in table1.find("thead").find("tr").findAll("td")]
print(headings1)
datasets1 = []
for row in table1.find_all("tr")[1:]:
    dataset1 = dict(zip(headings1, (td.get_text() for td in row.find_all("td"))))
    datasets1.append(dataset1)
df1=pd.DataFrame.from_dict(datasets1)

['Local name', 'Type', 'OSM', 'Address rank', 'Admin level', 'Distance', '']


In [11]:
df1.drop(df1[df1['Local name']=='No Name'].index,axis=0,inplace=True)
df1.drop(columns=['Type', 'OSM', 'Address rank', 'Admin level', 'Distance', ''],axis=1,inplace=True)
df1.drop_duplicates(inplace=True)
df1.drop(df1[df1['Local name'].isin( ['Gurugram','in','India','Haryana','Locality','Block A1','Village','Linked Places','Parent Of','Unclassified','Tertiary','Residential','Living_street','NH919','Trunk','Motorway','Drain','Bijwasan Drain','Peak','Administrative','Amer Road','Anthea','HUDA','Orange','Suburb','Town','Fort 2 - PedalYatri','Hamlet','Arawali Retreat','Delhi Western Peripheral Expressway',''])].index,inplace=True)
df1['Borough']='Gurgaon'
df1['lat']=np.NaN
df1['long']=np.NaN
df1.rename(columns={'Local name':'Neighbourhood'},inplace=True)
print(df1.shape)
df1.tail()

(219, 4)


,Neighbourhood,Borough,lat,long
284,Sector 70,Gurgaon,NaN,NaN
285,Sector 70A,Gurgaon,NaN,NaN
286,Sector 75,Gurgaon,NaN,NaN
287,Sector 76,Gurgaon,NaN,NaN
288,Sector 77,Gurgaon,NaN,NaN


In [12]:
geolocator1 = Nominatim()
indx=0
for g in df1.index:
    try:
        location1 = geolocator1.geocode(str(df1['Neighbourhood'][g]+' '+df1['Borough'][g]), timeout=5)
        #print(location1.latitude,location1.longitude)
        if location1.longitude:
            df1['lat'][g]=location1.latitude
            df1['long'][g]=location1.longitude
    except:
        indx=indx+1
        print('erorr in getting Lat/long for ', str(df1['Neighbourhood'][g]+';'+df1['Borough'][g]+';')) 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/i

In [13]:
print('total error in getting lat/long ',indx)
df1.head()

total error in getting lat/long  0


,Neighbourhood,Borough,lat,long
14,DLF I,Gurgaon,28.476056,77.102308
15,Hamilton Court,Gurgaon,28.467378,77.085082
16,Islampur,Gurgaon,28.401607,76.984521
17,Old DLF,Gurgaon,28.470145,77.043281
18,Regency Park II,Gurgaon,28.465812,77.086445


In [14]:
print('size of dataframe ', df1.shape)
print(df1.isnull().sum()) # To check if Lat/Long is not available 
gur_neighborhoods = df1.copy()
gur_neighborhoods.dropna(inplace=True) # Drop rows where Lat/Long is not available 
print('Size of DataFrame of Gurgaon neighborhoods along with Lat/Long ',gur_neighborhoods.shape)

size of dataframe  (219, 4)
Neighbourhood    0
Borough          0
lat              0
long             0
dtype: int64
Size of DataFrame of Gurgaon neighborhoods along with Lat/Long  (219, 4)


In [15]:
map_gur = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(gur_neighborhoods['lat'], gur_neighborhoods['long'], gur_neighborhoods['Borough'], gur_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_gur)
map_gur    

In [16]:
map_gur.save('map_gur.html')

Next, we will use Foursquare API to get the top 30 venues that are within a radius of 500 meters for each neighbourhoods. 
We need to register a Foursquare Developer Account in order to obtain the Foursquare ID and  Foursquare secret key. We then make API calls to Foursquare passing in the geographical coordinates of the neighbourhoods in a Python loop. Foursquare will return the venue data in JSON format and we will extract the venue name, venue category, venue latitude and longitude. With the data, we can check how many venues were returned for each neighbourhood and examine how many unique categories can be curated from all the returned venues. 

Then, we will analyse each neighbourhood by grouping the rows by neighbourhood and taking the mean of the frequency of occurrence of each venue category. By doing so, we are also preparing the data for use in clustering. Since we are analysing the “Shopping Mall” data, we will filter the “Shopping Mall” as venue category for the neighbourhoods. 


In [17]:
gur_venues = getNearbyVenues(names=gur_neighborhoods['Neighbourhood'],latitudes=gur_neighborhoods['lat'],longitudes=gur_neighborhoods['long'])
gur_venues.to_csv('gur_venues_1000radius_100limit_122001_nhburs1.csv')  

In [18]:
print('Gurgaon Venues dataframe size ',gur_venues.shape)

Gurgaon Venues dataframe size  (907, 7)


In [19]:
print('There are {} uniques categories.'.format(len(gur_venues['Venue Category'].unique())))
ven_cnt=gur_venues['Venue Category'].groupby(gur_venues['Venue Category']).count().sort_values(ascending=False)
ven_cnt.head(10)

There are 159 uniques categories.


Venue Category
Indian Restaurant       82
Hotel                   57
Café                    53
Pizza Place             36
Shopping Mall           34
Fast Food Restaurant    34
Coffee Shop             34
Restaurant              26
Chinese Restaurant      22
Brewery                 19
Name: Venue Category, dtype: int64

In [20]:
# one hot
gur_onehot = pd.get_dummies(gur_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
gur_onehot['Neighbourhood'] = gur_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [gur_onehot.columns[-1]] + list(gur_onehot.columns[:-1])
gur_onehot = gur_onehot[fixed_columns]

In [21]:
gur_grouped = gur_onehot.groupby('Neighbourhood').mean().reset_index()
print(gur_grouped.shape)
gur_grouped.head()

(140, 160)


,Neighbourhood,ATM,Advertising Agency,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,...,Theater,Tibetan Restaurant,Toll Booth,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Whisky Bar,Wine Bar
0,Aaronville,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abhyapur,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
2,Ambience Island,0.0,0.0,0.04878,0.02439,0.0,0.02439,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Araville,1.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar Phase III Extension,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = gur_grouped['Neighbourhood']

for ind in np.arange(gur_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(gur_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(140, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aaronville,Indian Restaurant,Brewery,Bakery,Pizza Place,Food Court,Department Store,Italian Restaurant,Hotel,Falafel Restaurant,Food & Drink Shop
1,Abhyapur,Campground,Volleyball Court,Wine Bar,Food Stand,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
2,Ambience Island,Fast Food Restaurant,Indian Restaurant,Restaurant,Shopping Mall,American Restaurant,Café,Multiplex,Gym,Italian Restaurant,Bar
3,Araville,ATM,Falafel Restaurant,Food Stand,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Electronics Store
4,Ashok Vihar Phase III Extension,Market,Beer Garden,Business Service,Moving Target,Farmers Market,Food Stand,Food Court,Food & Drink Shop,Food,Flea Market


In [23]:
print(gur_grouped[gur_grouped["Shopping Mall"] > 0][["Neighbourhood","Shopping Mall"]].shape)
gur_grouped[gur_grouped["Shopping Mall"] > 0][["Neighbourhood","Shopping Mall"]].head()

(26, 2)


,Neighbourhood,Shopping Mall
2,Ambience Island,0.048780
7,Central Park II,0.090909
9,DLF I,0.029412
19,Hamilton Court,0.017544
38,Sector 102,1.000000


In [24]:
kl_mall = gur_grouped[["Neighbourhood","Shopping Mall"]]

In [25]:
# set number of clusters
kclusters = 5

kl_clustering = kl_mall.drop(["Neighbourhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 2, 2, 2, 2, 0, 2, 2], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [27]:
#kl_merged.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighbourhood,Shopping Mall,Cluster Labels
0,Aaronville,0.00000,2
1,Abhyapur,0.00000,2
2,Ambience Island,0.04878,0
3,Araville,0.00000,2
4,Ashok Vihar Phase III Extension,0.00000,2


In [28]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(gur_neighborhoods.set_index("Neighbourhood"), on="Neighbourhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(140, 6)


,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
0,Aaronville,0.00000,2,Gurgaon,28.409919,77.037777
1,Abhyapur,0.00000,2,Gurgaon,28.294153,77.113791
2,Ambience Island,0.04878,0,Gurgaon,28.500780,77.102441
3,Araville,0.00000,2,Gurgaon,28.359185,76.975552
4,Ashok Vihar Phase III Extension,0.00000,2,Gurgaon,28.496716,77.027441


In [29]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.dropna(inplace=True) #drop rows where Lat/Long is not available
print(kl_merged.shape)
kl_merged

(140, 6)
(140, 6)


,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
69,Sector 26A,0.071429,0,Gurgaon,28.470387,77.099305
66,Sector 25,0.055556,0,Gurgaon,28.486089,77.084224
2,Ambience Island,0.048780,0,Gurgaon,28.500780,77.102441
65,Sector 24,0.062500,0,Gurgaon,28.494481,77.101304
97,Sector 51,0.100000,0,Gurgaon,28.428701,77.066688
...,...,...,...,...,...,...
88,Sector 43,0.200000,3,Gurgaon,28.454874,77.085886
59,Sector 2,0.166667,3,Gurgaon,28.509050,77.034283
63,Sector 23,0.333333,4,Gurgaon,28.510319,77.053009
119,Sector 82,0.250000,4,Gurgaon,28.393108,76.958876


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['lat'], kl_merged['long'], kl_merged['Neighbourhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
map_clusters.save('map_clusters.html')
kl_merged.loc[kl_merged['Cluster Labels'] == 0].sort_values('Shopping Mall',ascending=False)

,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
94,Sector 49,0.120000,0,Gurgaon,28.412885,77.049836
97,Sector 51,0.100000,0,Gurgaon,28.428701,77.066688
89,Sector 44,0.090909,0,Gurgaon,28.450706,77.073828
7,Central Park II,0.090909,0,Gurgaon,28.423428,77.035546
103,Sector 56,0.076923,0,Gurgaon,28.425323,77.098516
69,Sector 26A,0.071429,0,Gurgaon,28.470387,77.099305
65,Sector 24,0.062500,0,Gurgaon,28.494481,77.101304
66,Sector 25,0.055556,0,Gurgaon,28.486089,77.084224
2,Ambience Island,0.048780,0,Gurgaon,28.500780,77.102441


In [32]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1].sort_values('Shopping Mall')

,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
115,Sector 74,1.0,1,Gurgaon,28.411276,77.010669
116,Sector 75,1.0,1,Gurgaon,28.396448,77.006590
125,Sector 94,1.0,1,Gurgaon,28.421965,76.929771
127,Sihi,1.0,1,Gurgaon,28.411633,76.971110
38,Sector 102,1.0,1,Gurgaon,28.475487,76.971175
134,Tikri,1.0,1,Gurgaon,28.419725,77.035937


In [33]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2].sort_values('Shopping Mall')

,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
80,Sector 37C,0.000000,2,Gurgaon,28.448909,76.988050
39,Sector 104,0.000000,2,Gurgaon,28.479532,76.993720
6,Bilaspur Khurd,0.000000,2,Gurgaon,28.288275,76.870403
37,Sector 100,0.000000,2,Gurgaon,28.461390,76.971271
11,Damdama,0.000000,2,Gurgaon,28.307427,77.120553
...,...,...,...,...,...,...
130,Sunbreeze,0.000000,2,Gurgaon,28.397396,77.037756
114,Sector 72A,0.000000,2,Gurgaon,28.423323,77.018777
19,Hamilton Court,0.017544,2,Gurgaon,28.467378,77.085082
9,DLF I,0.029412,2,Gurgaon,28.476056,77.102308


In [34]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3].sort_values('Shopping Mall')

,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
96,Sector 50,0.133333,3,Gurgaon,28.416654,77.061175
101,Sector 54,0.166667,3,Gurgaon,28.440392,77.112079
59,Sector 2,0.166667,3,Gurgaon,28.509050,77.034283
139,Windsor Court,0.200000,3,Gurgaon,28.463775,77.085876
88,Sector 43,0.200000,3,Gurgaon,28.454874,77.085886


In [35]:
kl_merged.loc[kl_merged['Cluster Labels'] == 4].sort_values('Shopping Mall')

,Neighbourhood,Shopping Mall,Cluster Labels,Borough,lat,long
119,Sector 82,0.250000,4,Gurgaon,28.393108,76.958876
70,Sector 28,0.250000,4,Gurgaon,28.474130,77.082572
63,Sector 23,0.333333,4,Gurgaon,28.510319,77.053009


#### Results & Discussion

The results from the k-means clustering show that we can categorize the neighbourhoods into 5 
clusters based on the frequency of occurrence for “Shopping Mall”: 
- Cluster 0: Neighbourhoods low number of shopping malls 
- Cluster 1: Neighbourhoods with high numbers of shopping malls
- Cluster 2: Neighbourhoods very low numbers or non-existence of shopping malls 
- Cluster 3: Neighbourhoods with moderate concentration of shopping malls 
- Cluster 4: Neighbourhoods with moderate concentration of shopping malls 

The results of the clustering are visualized in the map below with cluster 0 in red colour, cluster 1 in 
purple colour, cluster 2 in light blue colour, cluster 3 in light green colour and cluster 4 in light orange colour.

Most of the shopping malls are scatter around the central part of the city, with the highest number in cluster 1 and moderate number in cluster 3/4. On the other hand, cluster 2 has very rare instances of Malls in these areas. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. 

Therefore, this project recommends real estate developers to capitalize on these findings to open new shopping malls in neighbourhoods in cluster 2 with little to no competition. 


#### Conclusion

Purpose of this project was to identify less/negligible numbers of Shopping Malls in Gurgaon city in order to guide real estate developers and city planner, with locations where a new Shopping Mall can be planned. So Cluster 2 areas plotted in map which are having most potential for laying a new shopping mall.